# 뉴럴 네트워크?

이번 강의에서 여러분은 현대 머신러닝의 시작지점인 뉴럴네트워크를 다루는 방법을 배운다.

In [2]:
# 전처리된 타이타닉 데이터셋 로드
import pandas as pd
titanic = pd.read_csv('clean_titanic.csv', index_col=0)

# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
encoders = {}
for col in titanic.columns:
    encoders[col] = LabelEncoder()
    encoders[col].fit(titanic[col])
    titanic[col] = encoders[col].transform(titanic[col])

# xs, ys 분리
xs, ys = titanic.drop(columns=['survived']), titanic['survived']

# tts
from sklearn.model_selection import train_test_split
xs_train, xs_test, ys_train, ys_test = train_test_split(xs, ys, test_size=0.2)

# 준비 끝!

In [ ]:
len(xs.columns)

## 쉬운 길

장난감 만들기

In [3]:
from sklearn.neural_network import MLPClassifier

## 11 -> 300 -> 300 -> 30 -> 1
clf = MLPClassifier(
    hidden_layer_sizes=(30, 30, 30),
    max_iter=300,
    activation='tanh',
    solver='adam',
)

clf.fit(xs_train, ys_train)
y_pred = clf.predict(xs_test)

# accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(ys_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.87


C:\Users\catac\AppData\Roaming\Python\Python313\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


여러분은 이제부터 Tensor 객체를 다루는 방법을 먼저 배워야 한다.

데이터를 Tensor(행렬)로 변환하고, 조작하고 등등의 기본 작업을 수행할 줄 알아야 한다.

## Tensor 선언

Tensor(행렬)은 PyTorch의 기본 데이터 구조이다.

PyTorch는 Tensor를 사용하여 데이터를 표현하고, 연산을 수행한다.

Tensor는 다차원 배열로, NumPy의 ndarray와 유사하며, 이것으로 수치 연산을 수행하고 딥러닝 모델을 구축한다.

Tensor는 CPU와 GPU에서 모두 사용할 수 있으며, GPU를 사용하여 연산을 가속화할 수 있다.

여기서 우리는 Tensor를 선언하는 기본적인 방법을 배우고, 기존 데이터를 Tensor로 변환할 것이다.

In [39]:
from torch import Tensor
from torch import ones, randn, zeros

print(Tensor([[2, 3, 3],
              [4, 5, -1]]).shape)

torch.Size([2, 3])


In [37]:
print(Tensor([1]).shape)
print(Tensor([[1]]).shape)
print(Tensor([[[1]]]).shape)

torch.Size([1])
torch.Size([1, 1])
torch.Size([1, 1, 1])


In [40]:
# 2x2 tensor 생성
print(Tensor([[1, 2],
              [3, 4]]))

# 4x2 tensor 생성
print(Tensor([[1, 2],
              [3, 4],
              [5, 6],
              [7, 8]]))

tensor([[1., 2.],
        [3., 4.]])
tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])


계속 이렇게 만들어야 하나요?

**네**

In [50]:
# 빈 tensor 만들기
from torch import zeros, ones, randn

# 2x2 tensor zerofill
print(zeros(2, 2))

tensor([[0., 0.],
        [0., 0.]])


In [51]:
# 1로 채워진 tensor 만들기
print(ones(2, 2))

# 4x2x6 tensor 생성
print(ones(4, 2, 6))

tensor([[1., 1.],
        [1., 1.]])
tensor([[[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]]])


In [53]:
# 랜덤한 값으로 채워진 tensor 만들기
print(randn(3, 6))

tensor([[-0.1164, -1.8869,  0.1259, -1.5040, -0.3281, -0.1288],
        [-1.5946,  0.5637, -1.7794, -0.9208, -0.6846,  0.6163],
        [-0.2088, -0.8003, -0.9282,  2.3892, -0.9549,  0.8996]])


In [61]:
print(type(xs_train))
print(type(xs_train.values))
print(type(Tensor(xs_train.values)))

Tensor(xs_train.values)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'torch.Tensor'>


tensor([[ 2.,  1., 36.,  ...,  1.,  2.,  1.],
        [ 0.,  1., 78.,  ...,  1.,  2.,  1.],
        [ 2.,  0., 39.,  ...,  0.,  1.,  1.],
        ...,
        [ 0.,  1., 65.,  ...,  1.,  0.,  0.],
        [ 2.,  1., 24.,  ...,  1.,  2.,  1.],
        [ 2.,  1., 29.,  ...,  1.,  2.,  1.]])

## 텐서 연산

두 텐서간의 연산을 지원한다.

In [70]:
t1 = Tensor([[1, 2],
             [3, 4]])
t1

tensor([[1., 2.],
        [3., 4.]])

In [71]:
# 스칼라 덧셈, 뺄셈, 곱셈, 나눗셈
t1 + 1, t1 - 1, t1 * 2, t1 / 2

(tensor([[2., 3.],
         [4., 5.]]),
 tensor([[0., 1.],
         [2., 3.]]),
 tensor([[2., 4.],
         [6., 8.]]),
 tensor([[0.5000, 1.0000],
         [1.5000, 2.0000]]))

In [77]:
vec = Tensor([-2])

t1 / vec

tensor([[-0.5000, -1.0000],
        [-1.5000, -2.0000]])

In [86]:
# 벡터곱
vec = Tensor([1, -1])

t2 = Tensor([[1, 2, 3],
             [4, 5, 6]])
t3 = Tensor([1, 2, 3])
t2.shape, t3.shape

(torch.Size([2, 3]), torch.Size([3]))

In [89]:
# 같은 크기
t2 = Tensor([[1, -1],
             [2, -2]])

t1 / t2

tensor([[ 1.0000, -2.0000],
        [ 1.5000, -2.0000]])

In [97]:
# 컴까기
t4 = ones(4, 2, 3)
t4.shape

torch.Size([4, 2, 3])

In [99]:
t5 = Tensor([[[1, 2, 3],
             [4, 5, 6]],

             [[-1, -2, -3],
             [-4, -5, -6]]])
t5.shape

torch.Size([2, 2, 3])

In [128]:
(t4.reshape(-1, 2, 2, 3) * t5).reshape(-1, 2, 3).shape

torch.Size([4, 2, 3])

In [132]:
t6 = ones(7)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [ ]:
t1.matmul(t2)  # 행렬 곱셈

In [ ]:
# 다른 크기
t3 = Tensor([[1, 2],
             [3, 4],
             [5, 6],
             [7, 8]])

t1 + t3, t1 - t3, t1 * t3, t1 / t3  # 브로드캐스팅 실패

## `Tensor.reshape(d, i, m, e, n, s, i, o, n)`

Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
중요함

In [101]:
# reshape (재구성)
t = Tensor([[1, 2],
            [3, 4],
            [5, 6]])
print(t)

tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])


In [102]:
t.flatten()  # 1차원으로 평탄화

tensor([1., 2., 3., 4., 5., 6.])

In [107]:
t.reshape(1, 2, 1, 2, 3)

RuntimeError: shape '[1, 2, 1, 2, 3]' is invalid for input of size 6

In [117]:
t = ones(3, 4, 5, 2, 3)
t.reshape(10, -1).shape

torch.Size([10, 36])

In [118]:
t.reshape(3, -1)  # 3행, 열은 되는데로

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.

In [135]:
xs_train_tensor = Tensor(xs_train.values)
xs_test_tensor = Tensor(xs_test.values)
ys_train_tensor = Tensor(ys_train.values)
ys_test_tensor = Tensor(ys_test.values)

In [141]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(
    xs_train_tensor, ys_train_tensor
)  # 훈련용 x, y값을 묶어서 데이터셋 생성

for_train = DataLoader(
    train_dataset, batch_size=32, shuffle=True
)

## Practice 1.

1. `clean_?` 데이터셋에서 분리한 xs_train, xs_test, ys_train, ys_test를 각각 Tensor로 변환한다.
2. xs_train, ys_train을 DataLoader에 전달하여 배치 크기 32짜리 훈련 데이터셋을 만든다.

## 네트워크 생성

이제 여러분은 Tensor를 다루는 방법을 배웠다.

이제는 뉴럴 네트워크를 생성하는 방법을 배워야 한다.

이 단계에서 배우는 두 가지 개념은 **레이어**와 **층**이다.

In [150]:
from torch import nn

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

class TitanicModel(nn.Module):
    def __init__(self):
        super(TitanicModel, self).__init__()

        self.a = nn.Linear(11, 30)
        self.b = nn.Linear(30, 20)
        self.c = nn.Linear(20, 1)

tm = TitanicModel()
tm

TitanicModel(
  (a): Linear(in_features=11, out_features=30, bias=True)
  (b): Linear(in_features=30, out_features=20, bias=True)
  (c): Linear(in_features=20, out_features=1, bias=True)
)

In [ ]:
# 4개의 값을 받아 1개의 값을 출력하는 모델을 정의한다.
# 4->10->10->1 총 3층으로 이뤄진 모델

class ModelAlpha(nn.Module):
    def __init__(self):
        super(ModelAlpha, self).__init__()

        self.lin_input = nn.Linear(4, 10)
        self.lin_2 = nn.Linear(10, 10)
        self.lin_output = nn.Linear(10, 1)


In [ ]:
# 4개 값을 받아 3개의 값을 출력하는 모델을 정의한다.
# 4->20->30->20->3 총 4층으로 이뤄진 모델

class ModelBravo(nn.Module):
    def __init__(self):
        super(ModelBravo, self).__init__()

        self.l1 = nn.Linear(4, 20)
        self.l2 = nn.Linear(20, 30)
        self.l3 = nn.Linear(30, 20)
        self.l4 = nn.Linear(20, 3)


## Practice 2. 빈 모델 만들기

1. `ModelCharlie` 클래스를 입력층 5, 은닉층 10, 10, 출력층 1로 네트워크를 구성해 만드세요.
2. `ModelDelta` 클래스를 입력층 3, 은닉층 6, 12, 24, 12, 6, 3, 출력층 1로 네트워크를 구성해 만드세요.
3. `ModelEcho` 클래스를 입력층 10, 은닉층 모두 30, 출력층 1로 구성된 총 51층 네트워크를 구성해 만들 방법을 생각해보세요.
(힌트 - 언패킹)


In [153]:
class ModelCharlie(nn.Module):
    def __init__(self):
        super(ModelCharlie, self).__init__()
        num_neurons = [5, 10, 10, 1]
        layers = []
        for i, n in enumerate(num_neurons[:-1]):
            _in = num_neurons[i]
            _out = num_neurons[i + 1]
            layers.append(nn.Linear(_in, _out))

        self.a, self.b, self.c = layers

mc = ModelCharlie()
mc

ModelCharlie(
  (a): Linear(in_features=5, out_features=10, bias=True)
  (b): Linear(in_features=10, out_features=10, bias=True)
  (c): Linear(in_features=10, out_features=1, bias=True)
)

In [154]:
class ModelDelta(nn.Module):
    def __init__(self):
        super(ModelDelta, self).__init__()
        num_neurons = [3, 6, 12, 24, 12, 6, 3, 1]
        layers = []
        for i, n in enumerate(num_neurons[:-1]):
            _in = num_neurons[i]
            _out = num_neurons[i + 1]
            layers.append(nn.Linear(_in, _out))

        self.layer = nn.Sequential(*layers)

md = ModelDelta()
md

ModelDelta(
  (layer): Sequential(
    (0): Linear(in_features=3, out_features=6, bias=True)
    (1): Linear(in_features=6, out_features=12, bias=True)
    (2): Linear(in_features=12, out_features=24, bias=True)
    (3): Linear(in_features=24, out_features=12, bias=True)
    (4): Linear(in_features=12, out_features=6, bias=True)
    (5): Linear(in_features=6, out_features=3, bias=True)
    (6): Linear(in_features=3, out_features=1, bias=True)
  )
)

In [158]:
class ModelEcho(nn.Module):
    def __init__(self):
        super(ModelEcho, self).__init__()
        num_neurons = [10] + [30]*49 + [1]
        layers = []
        for i, n in enumerate(num_neurons[:-1]):
            _in = num_neurons[i]
            _out = num_neurons[i + 1]
            layers.append(nn.Linear(_in, _out))

        self.layer = nn.Sequential(*layers)

me = ModelEcho()
me

ModelEcho(
  (layer): Sequential(
    (0): Linear(in_features=10, out_features=30, bias=True)
    (1): Linear(in_features=30, out_features=30, bias=True)
    (2): Linear(in_features=30, out_features=30, bias=True)
    (3): Linear(in_features=30, out_features=30, bias=True)
    (4): Linear(in_features=30, out_features=30, bias=True)
    (5): Linear(in_features=30, out_features=30, bias=True)
    (6): Linear(in_features=30, out_features=30, bias=True)
    (7): Linear(in_features=30, out_features=30, bias=True)
    (8): Linear(in_features=30, out_features=30, bias=True)
    (9): Linear(in_features=30, out_features=30, bias=True)
    (10): Linear(in_features=30, out_features=30, bias=True)
    (11): Linear(in_features=30, out_features=30, bias=True)
    (12): Linear(in_features=30, out_features=30, bias=True)
    (13): Linear(in_features=30, out_features=30, bias=True)
    (14): Linear(in_features=30, out_features=30, bias=True)
    (15): Linear(in_features=30, out_features=30, bias=True)


## 순전파와 활성함수

뉴럴네트워크 모델은 x값을 받아 y값을 계산하는 방법을 지정해주어야 한다.

In [ ]:
class ModelFoxtrot(nn.Module):
    def __init__(self):
        super(ModelFoxtrot, self).__init__()
        self.lin1 = nn.Linear(4, 10)  # 입력층: 4개의 특성, 은닉층: 10개의 뉴런
        self.lin2 = nn.Linear(10, 10)  # 은닉층: 10개의 뉴런
        self.lin3 = nn.Linear(10, 1)   # 출력층: 1개의 뉴런

        self.activation1 = nn.ReLU()
        self.activation2 = nn.Sigmoid()
        self.activation4 = nn.LeakyReLU()
        self.activation3 = nn.Tanh()
        self.activation5 = nn.Sigmoid()
        self.activation6 = nn.SELU()
        self.activation7 = nn.ELU()
        self.activation8 = nn.GELU()


    def forward(self, x):
        x = self.lin1(x)
        x = self.activation1(x)
        x = self.lin2(x)
        x = self.activation2(x)
        return self.lin3(x)



### 활성함수

1. ReLU (Rectified Linear Unit): `f(x) = max(0, x)`
2. Sigmoid: `f(x) = 1 / (1 + exp(-x))`
3. Tanh (Hyperbolic Tangent): `f(x) = (exp(x) - exp(-x)) / (exp(x) + exp(-x))`
4. Leaky ReLU: `f(x) = x if x > 0 else alpha * x` (alpha는 작은 상수)
5. SELU (Scaled Exponential Linear Unit): `f(x) = scale * x if x > 0 else scale * alpha * (exp(x) - 1)`
6. ELU (Exponential Linear Unit): `f(x) = x if x > 0 else alpha * (exp(x) - 1)`
7. GELU (Gaussian Error Linear Unit): `f(x) = 0.5 * x * (1 + erf(x / sqrt(2)))` (erf는 오차 함수)
8. Swish: `f(x) = x * sigmoid(beta * x)` (beta는 상수)

이 외에도 활성함수가 여럿 있다. 이들이 각각 어디에 쓰이는지는 목적상황에 따라 다르다.

## Practice 3. 순전파 구현하기

1. `ModelGolf` (5, 6, 3) 네트워크와 ReLU 활성함수를 사용한 순전파를 구현하세요.
2. `ModelHotel` (3, 6, 9, 12, 1) 네트워크와 LeakyReLU 활성함수를 사용한 순전파를 구현하세요.
3. `ModelIndia` (10, 20, 30, 20, 10, 5, 1) 네트워크와 여러가지 활성함수를 사용한 순전파를 구현하세요.

## 훈련함수

모델을 정의했다면, 이제 예측함수를 구현해야 한다.


##